In [7]:
from flask import Flask, request, jsonify, render_template, url_for, redirect
from flask_cors import CORS
from flasgger import Swagger
import json
from utility.DBConInfo import dbConInfo
from utility.Env import env
from utility.log import log_record

import logging
from logging.handlers import TimedRotatingFileHandler
import os
if not os.path.isdir('log'):
    os.makedirs('log', exist_ok=True)

In [8]:
app = Flask(__name__)
CORS(app)
Swagger(app)

@app.route("/available_table",methods=['POST'])
def available_table():
    """
    取得資料庫中所有的table名稱
    調用此API即可獲取public資料庫中所有table名稱
    ---
    tags:
      - available table
    responses:
      200:
        description: Success
        schema:
          id: available table return
          properties:
            status:
              type: string
              description: 錯誤碼
            result:
              type: array
              description: table name
              items:
                  type: string
    """

    try:
        dci=dbConInfo(env.host(),env.user(),env.password(),env.dbname(),env.sslmode())
        sql="SELECT * FROM pg_catalog.pg_tables"
        logger.info(f'execute : {sql}')
        df=dci.readSQL(sql)
        df_public=df[df.schemaname=="public"]
        list_public=df_public.tablename.values.tolist()
        return json.dumps({"status":"success","result":list_public})
    
    except Exception as e:
        logger.error('[error] route : available_table ; message : '+ str(e))
        return json.dumps({"status":"error","result":str(e)})
        
@app.route("/query_table",methods=['POST'])
def query_table():
    """
    查詢指定table中的資料
    調用此API即可獲取指定table中指定欄位的資料
    ---
    tags:
      - query table
    
    parameters:
      - name: index
        in: formData
        type: string
        required: true
        description: table名稱
        x-example: mes-eim
      - name: column
        in: formData
        type: string
        description: 可選擇資料欄位，若沒回傳參數defalut為*
        x-example: '*'
      - name: active
        in: formData
        type: string
        description: active欄位中的內容-Y/N，若沒回傳參數defalut為""
        x-example: 'Y'
    responses:
      200:
        description: Success
        schema:
          id: query table return
          properties:
            status:
              type: string
              description: 錯誤碼
            result:
              type: array
              description: column header andcontain by row
              items:
                  type: string
    """
    try:
        dci=dbConInfo(env.host(),env.user(),env.password(),env.dbname(),env.sslmode())
        index = request.form.get('index')
        column = request.form.get('column') if request.form.get('column') else '*'
        active = "where active='Y'" if request.form.get('active')=="Y" else ''
        sql=f"SELECT {column} FROM public.{index} {active}"
        logger.info(f'execute : {sql}')
        df=dci.readSQL(sql)
        df_header = df.columns.values.tolist()
        df_list = df.values.tolist()
        df_list.insert(0, df_header)
        return json.dumps({"status":"success","result":df_list})

    except Exception as e:
        logger.error('route : query_table ; message : '+ str(e))
        return json.dumps({"status":"error","result":str(e)})

In [ ]:
if __name__ == "__main__":
    
    logger = logging.getLogger("db_api")
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt = "%Y-%m-%d %H:%M:%S")
    handler = TimedRotatingFileHandler(f'log/db_api.log', when="midnight")
    handler.setFormatter(formatter)
    handler.suffix = "%Y%m%d"
    logger.addHandler(handler)
    
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
192.168.153.167 - - [17/Jun/2022 16:34:21] "POST /query_table HTTP/1.1" 200 -
